https://medium.com/@rubentak/unleashing-the-power-of-intelligent-chatbots-with-gpt-4-and-vector-databases-a-step-by-step-8027e2ce9e78

In [1]:
import dotenv
import openai
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, CSVLoader

In [2]:
dotenv.load_dotenv('.env')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Documents

In [3]:
import pandas as pd

df = pd.read_csv('../../mmm/docs/doc_mmm.csv', sep=';')
df

,FUNCIÓN,OBJETIVO,ARGUMENTOS,RETURN,EJEMPLO
0,R2(),Calcula la bondad del ajuste de un modelo esta...,fit : sm.regression.linear_model.RegressionRes...,float\nEl coeficiente de determinación R² del ...,NaN
1,calculate_geom_ad_stock,Calcula el 'ad stock' geométrico de una serie ...,series : List[float]\nLista de valores que rep...,List[float]\nLista de valores actualizados rep...,NaN


In [4]:
loader = CSVLoader('../../mmm/docs/doc_mmm.csv', encoding="utf-8", csv_args={'delimiter': ';'})
data = loader.load()

In [5]:
# Print number of txt files in directory
# loader = DirectoryLoader('../../mmm/html/mmm', glob="./*.html")
loader = DirectoryLoader(
    '../../mmm/docs', 
    loader_cls=CSVLoader, 
    loader_kwargs={'encoding': 'utf-8', 'csv_args': {'delimiter': ';'}}
    )
doc = loader.load ( )
len(doc)

2

In [6]:
# Splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size    = 1000, 
    chunk_overlap = 200
    )
texts = text_splitter.split_documents(doc)


# Count the number of chunks
len(texts)

2

In [7]:
texts

[Document(page_content='\ufeffFUNCIÓN: R2()\nOBJETIVO: Calcula la bondad del ajuste de un modelo estadístico.\n\nEste método calcula el coeficiente de determinación (R²), que es una medida de qué tan bien las variables independientes predicen la variable dependiente. Un R² más alto indica un mejor ajuste del modelo.\nARGUMENTOS: fit :\u2002sm.regression.linear_model.RegressionResultsWrapper\nUn modelo ajustado utilizando la biblioteca statsmodels. Este objeto contiene tanto los residuos como la variable dependiente del modelo.\nRETURN: float\nEl coeficiente de determinación R² del modelo.\nEJEMPLO:', metadata={'source': '..\\..\\mmm\\docs\\doc_mmm.csv', 'row': 0}),
 Document(page_content="\ufeffFUNCIÓN: calculate_geom_ad_stock\nOBJETIVO: Calcula el 'ad stock' geométrico de una serie de datos aplicando un factor de decaimiento.\n\nEl 'ad stock' es un concepto en marketing y economía que representa el impacto acumulado de la publicidad a lo largo del tiempo. Esta función aplica un modelo

In [8]:
print(texts[1].page_content)

FUNCIÓN: calculate_geom_ad_stock
OBJETIVO: Calcula el 'ad stock' geométrico de una serie de datos aplicando un factor de decaimiento.

El 'ad stock' es un concepto en marketing y economía que representa el impacto acumulado de la publicidad a lo largo del tiempo. Esta función aplica un modelo de ad stock geométrico, donde el efecto de la publicidad se deprecia de forma geométrica en cada período.
ARGUMENTOS: series : List[float]
Lista de valores que representan la inversión o impacto publicitario en cada período.
decay_factor : float
Factor de decaimiento que se aplica a la acumulación de ad stock. Si es mayor que 1, se considera como porcentaje y se divide por 100.
initial_value : float, optional
Valor inicial de ad stock. Por defecto es 0.
RETURN: List[float]
Lista de valores actualizados representando el ad stock acumulado.
EJEMPLO:


# ChromaDB Database

In [4]:
persist_directory = 'db'
docs_directory    = '../../mmm/docs'

# OpenAI embeddings
embedding = OpenAIEmbeddings()

if os.path.exists(persist_directory):
    # Load the persisted database from disk
    print("Loading existing Vector DB")
    vectordb = Chroma(
        persist_directory  = persist_directory,
        embedding_function = embedding
        )
else:
    print("Creating VectorDB")

    # Load text in docs
    loader = DirectoryLoader(
        docs_directory, 
        loader_cls=CSVLoader, 
        loader_kwargs={'encoding': 'utf-8', 'csv_args': {'delimiter': ';'}}
    )
    doc = loader.load()

    # Splitting the text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size    = 1000, 
        chunk_overlap = 200
        )
    texts = text_splitter.split_documents(doc)

    # Put the text chunks into embeddings in a local Chroma vector database.
    # Supplying a persist_directory will store the embeddings on the disk.
    vectordb = Chroma.from_documents(
        documents         = texts,
        embedding         = embedding,
        persist_directory = persist_directory
        )
    
    # Persist the db to disk
    vectordb.persist()

Creating VectorDB


In [6]:
texts


[Document(page_content='\ufeffFUNCIÓN: R2()\nOBJETIVO: Calcula la bondad del ajuste de un modelo estadístico.\n\nEste método calcula el coeficiente de determinación (R²), que es una medida de qué tan bien las variables independientes predicen la variable dependiente. Un R² más alto indica un mejor ajuste del modelo.\nARGUMENTOS: fit :\u2002sm.regression.linear_model.RegressionResultsWrapper\nUn modelo ajustado utilizando la biblioteca statsmodels. Este objeto contiene tanto los residuos como la variable dependiente del modelo.\nRETURN: float\nEl coeficiente de determinación R² del modelo.\nEJEMPLO:', metadata={'source': '..\\..\\mmm\\docs\\doc_mmm.csv', 'row': 0}),
 Document(page_content="\ufeffFUNCIÓN: calculate_geom_ad_stock\nOBJETIVO: Calcula el 'ad stock' geométrico de una serie de datos aplicando un factor de decaimiento.\n\nEl 'ad stock' es un concepto en marketing y economía que representa el impacto acumulado de la publicidad a lo largo del tiempo. Esta función aplica un modelo

## Retriever

In [12]:
# Create retriever
retriever = vectordb.as_retriever(
    # search_type   = 'mmr',
    search_kwargs = {"k": 1},
    # n_results = 1
    )

docs = retriever.get_relevant_documents("Qué función calcula el ad stock?")

In [13]:
[print(x.page_content + "\n\n##########\n\n") for x in docs]

FUNCIÓN: calculate_geom_ad_stock
OBJETIVO: Calcula el 'ad stock' geométrico de una serie de datos aplicando un factor de decaimiento.

El 'ad stock' es un concepto en marketing y economía que representa el impacto acumulado de la publicidad a lo largo del tiempo. Esta función aplica un modelo de ad stock geométrico, donde el efecto de la publicidad se deprecia de forma geométrica en cada período.
ARGUMENTOS: series : List[float]
Lista de valores que representan la inversión o impacto publicitario en cada período.
decay_factor : float
Factor de decaimiento que se aplica a la acumulación de ad stock. Si es mayor que 1, se considera como porcentaje y se divide por 100.
initial_value : float, optional
Valor inicial de ad stock. Por defecto es 0.
RETURN: List[float]
Lista de valores actualizados representando el ad stock acumulado.
EJEMPLO:

##########




[None]

In [14]:
docs = retriever.get_relevant_documents("Qué función calcula el coeficiente de determinacion?")
docs


[Document(page_content='\ufeffFUNCIÓN: R2()\nOBJETIVO: Calcula la bondad del ajuste de un modelo estadístico.\n\nEste método calcula el coeficiente de determinación (R²), que es una medida de qué tan bien las variables independientes predicen la variable dependiente. Un R² más alto indica un mejor ajuste del modelo.\nARGUMENTOS: fit :\u2002sm.regression.linear_model.RegressionResultsWrapper\nUn modelo ajustado utilizando la biblioteca statsmodels. Este objeto contiene tanto los residuos como la variable dependiente del modelo.\nRETURN: float\nEl coeficiente de determinación R² del modelo.\nEJEMPLO:', metadata={'row': 0, 'source': '..\\..\\mmm\\docs\\doc_mmm.csv'})]

In [15]:
[print(x.page_content + "\n\n##########\n\n") for x in docs]

FUNCIÓN: R2()
OBJETIVO: Calcula la bondad del ajuste de un modelo estadístico.

Este método calcula el coeficiente de determinación (R²), que es una medida de qué tan bien las variables independientes predicen la variable dependiente. Un R² más alto indica un mejor ajuste del modelo.
ARGUMENTOS: fit : sm.regression.linear_model.RegressionResultsWrapper
Un modelo ajustado utilizando la biblioteca statsmodels. Este objeto contiene tanto los residuos como la variable dependiente del modelo.
RETURN: float
El coeficiente de determinación R² del modelo.
EJEMPLO:

##########




[None]

# Delete Chroma DB

In [7]:
vectordb.delete_collection()
vectordb.persist()

ValueError: Collection langchain does not exist.

In [8]:
# delete the directory
!rm -rf db/

rm: cannot remove 'db/chroma.sqlite3': Device or resource busy


# TEST

In [2]:
from utils.docretriever import DocRetriever

persist_directory = 'db'
docs_directory    = '../../mmm/docs'

doc_retriever = DocRetriever(
    persist_directory = persist_directory, 
    docs_directory    = docs_directory,
    loader_kwargs = {'encoding': 'utf-8', 'csv_args': {'delimiter': ';'}}
)

Loading existing Vector DB


In [13]:
print(doc_retriever
      .retriever
      .get_relevant_documents("Qué función calcula el ad stock?")[0]
      .page_content)

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


FUNCIÓN: calculate_geom_ad_stock
OBJETIVO: Calcula el 'ad stock' geométrico de una serie de datos aplicando un factor de decaimiento.

El 'ad stock' es un concepto en marketing y economía que representa el impacto acumulado de la publicidad a lo largo del tiempo. Esta función aplica un modelo de ad stock geométrico, donde el efecto de la publicidad se deprecia de forma geométrica en cada período.
ARGUMENTOS: series : List[float]
Lista de valores que representan la inversión o impacto publicitario en cada período.
decay_factor : float
Factor de decaimiento que se aplica a la acumulación de ad stock. Si es mayor que 1, se considera como porcentaje y se divide por 100.
initial_value : float, optional
Valor inicial de ad stock. Por defecto es 0.
RETURN: List[float]
Lista de valores actualizados representando el ad stock acumulado.
EJEMPLO:


In [14]:
print(doc_retriever
      .retriever
      .get_relevant_documents("Qué función calcula el coeficiente de determinacion?")[0]
      .page_content)

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


FUNCIÓN: R2()
OBJETIVO: Calcula la bondad del ajuste de un modelo estadístico.

Este método calcula el coeficiente de determinación (R²), que es una medida de qué tan bien las variables independientes predicen la variable dependiente. Un R² más alto indica un mejor ajuste del modelo.
ARGUMENTOS: fit : sm.regression.linear_model.RegressionResultsWrapper
Un modelo ajustado utilizando la biblioteca statsmodels. Este objeto contiene tanto los residuos como la variable dependiente del modelo.
RETURN: float
El coeficiente de determinación R² del modelo.
EJEMPLO:


In [3]:
doc_retriever = DocRetriever(
    persist_directory = persist_directory, 
    docs_directory    = docs_directory,
    k = 2,
    loader_kwargs = {'encoding': 'utf-8', 'csv_args': {'delimiter': ';'}}
)

Loading existing Vector DB


In [11]:
print([print(x.page_content + "\n\n########\n\n") for x in doc_retriever
      .retriever
      .get_relevant_documents("Aplica el adstock a las variables numericas, ajusta el modelo y calcula su R2")])


Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


FUNCIÓN: R2()
OBJETIVO: Calcula la bondad del ajuste de un modelo estadístico.

Este método calcula el coeficiente de determinación (R²), que es una medida de qué tan bien las variables independientes predicen la variable dependiente. Un R² más alto indica un mejor ajuste del modelo.
ARGUMENTOS: fit : sm.regression.linear_model.RegressionResultsWrapper
Un modelo ajustado utilizando la biblioteca statsmodels. Este objeto contiene tanto los residuos como la variable dependiente del modelo.
RETURN: float
El coeficiente de determinación R² del modelo.
EJEMPLO:

########


﻿FUNCIÓN: calculate_geom_ad_stock
OBJETIVO: Calcula el 'ad stock' geométrico de una serie de datos aplicando un factor de decaimiento.

El 'ad stock' es un concepto en marketing y economía que representa el impacto acumulado de la publicidad a lo largo del tiempo. Esta función aplica un modelo de ad stock geométrico, donde el efecto de la publicidad se deprecia de forma geométrica en cada período.
ARGUMENTOS: series : Lis